# Additional End of week Exercise - week 2

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

In [1]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [3]:
MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
openai = OpenAI()


OpenAI API Key exists and begins sk-proj-


In [16]:
system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

In [17]:
ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499", "san diego": "$999", "norfolk": "$12"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

In [18]:
# There's a particular dictionary structure that's required to describe our function:

price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [19]:
durations = {"london": "13 hours", "paris": "15 hours", "tokyo": "23 hours", "berlin": "8 hours", "san diego": "16 hours", "norfolk": "5 minutes"}

def get_duration(destination_city):
    print(f"TOOL: getting duration for {destination_city}")
    city = destination_city.lower()
    return durations.get(city, "Unknown")

In [44]:
duration_function = {
    "name": "get_duration",
    "description": "Returns the hours and minutes it takes to travel to a destination city",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "the city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

functions = [
    price_function,
    duration_function
]

In [50]:
#tools = [
#    {"type": "function", "function": price_function},
#    {"type": "function", "function": duration_function}
#]
functions = [
    price_function,
    duration_function
]

In [57]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    #TEMP: response = openai.chat.completions.create(model=MODEL_GPT, messages=messages, tools=tools)
    response = openai.chat.completions.create(model=MODEL_GPT, messages=messages, functions=functions, function_call="auto")

    print(f"chat() :: Finish Reason: {response.choices[0].finish_reason}")
    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response, city = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=MODEL_GPT, messages=messages)
    
    return response.choices[0].message.content

In [56]:
# We have to write that function handle_tool_call:

def handle_tool_call(message):
    print(f"handle_tool_call() :: tool_calls: {message.tool_calls}")
    responses = []
    for tool_call in message.tool_calls:
        arguments = json.loads(tool_call.function.arguments)
        if tool_call.name == "get_ticket_price":
            city = arguments.get('destination_city')
            price = get_ticket_price(city)
            response = {
                "role": "tool",
                "content": json.dumps({"destination_city": city,"price": price}),
                "tool_call_id": tool_call.id
            }
            responses.append(response)
        elif tool_call.name == "get_duration":
            city = arguments.get('destination_city')
            duration = get_duration(city)
            response = {
                "role": "tool",
                "content": json.dumps({"destination_city": city,"duration": duration}),
                "tool_call_id": tool_call.id
            }
            responses.append(response)
    
    return responses, city

In [55]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7906

To create a public link, set `share=True` in `launch()`.


Finish Reason: stop
Finish Reason: function_call


Traceback (most recent call last):
  File "/opt/anaconda3/envs/llms/lib/python3.11/site-packages/gradio/queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/envs/llms/lib/python3.11/site-packages/gradio/route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/envs/llms/lib/python3.11/site-packages/gradio/blocks.py", line 2103, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/envs/llms/lib/python3.11/site-packages/gradio/blocks.py", line 1648, in call_function
    prediction = await fn(*processed_input)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/envs/llms/lib/python3.11/site-packages/gradio/utils.py", line 857, in async_wrapper
    response = await f(*args, **kwargs)
         